In [14]:
seed = 1
import numpy as np

def seed_torch(seed=1029):
    np.random.seed(seed)
    torch.manual_seed(seed)



In [15]:
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import torch.nn as nn
class Net(nn.Module):
    def __init__(self, D_in, H1, H2, H3, D_out):
 
        super(Net, self).__init__()
        self.linear1 = nn.Linear(D_in, H1) 
        self.linear2 = nn.Linear(H1, H2) 
        self.linear3 = nn.Linear(H2, H3) 
        self.linear4 = nn.Linear(H3, D_out)
  
    def forward(self, x):
    
        x = torch.sigmoid(self.linear1(x))
        x = torch.sigmoid(self.linear2(x))
        x = torch.sigmoid(self.linear3(x))
        x = self.linear4(x)
        return x



In [16]:
import copy
import numpy as np
seed_torch(seed)
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H1, H2, H3, D_out = 50, 5, 5,10, 10, 3
model = Net(D_in, H1, H2, H3, D_out)
loss_fn = torch.nn.MSELoss(reduction='sum')

init_state = copy.deepcopy(model.state_dict())



In [17]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from tqdm import tqdm

import xlrd
import pandas as pd
book = xlrd.open_workbook('dataset for standardisation.xlsx')
sheet = book.sheet_by_name('Sheet1')
data_unreplicated = [[sheet.cell_value(r, c)
         for c in range(1,9)] for r in range(2,74)]
scaler = StandardScaler()
scaler.fit(data_unreplicated)
book = xlrd.open_workbook('prepareddatakfold.xlsx')
sheet = book.sheet_by_name('Sheet1')
data_replicated = [[sheet.cell_value(r, c)
         for c in range(0,8)] for r in range(0,7200)]

scaler = StandardScaler()
scaler.fit(data_unreplicated)
standardised_data = scaler.transform(data_replicated)

x_train = standardised_data[:, 0:5]
y_train = standardised_data[:, 5:8]

n_splits = 6
splits = list(KFold(n_splits=n_splits, shuffle=False, random_state=seed)
              .split(x_train, y_train))



In [19]:
i = n_splits-1
batches = 50
learning_rate = 5e-4
for q in range (5,25,5):
    for l in range (5,25,5):
        for m in range (3,15,3):
            N, D_in, H1, H2, H3, D_out = batches, 5, q, l , m, 3
            model = Net(D_in, H1, H2, H3, D_out)
            model.train()
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
            init_state = copy.deepcopy(model.state_dict())
            init_state_opt = copy.deepcopy(optimizer.state_dict())
            av_opt_epoch=0
            av_opt_loss=0
            for i, (train_idx, valid_idx) in enumerate(splits):
                model.load_state_dict(init_state)
                optimizer.load_state_dict(init_state_opt)
                x_train_fold = torch.tensor(x_train[train_idx], dtype=torch.float32)
                y_train_fold = torch.tensor(y_train[train_idx], dtype=torch.float32)
                x_val_fold = torch.tensor(x_train[valid_idx], dtype=torch.float32)
                y_val_fold = torch.tensor(y_train[valid_idx], dtype=torch.float32)
                train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
                valid = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)
                train_loader = torch.utils.data.DataLoader(train, batch_size=batches,
                                                           shuffle=True)
                valid_loader = torch.utils.data.DataLoader(valid, batch_size=batches,
                                                           shuffle=False)
                UP=0
                epoch=0
                k=0
                opt_loss=1000
                old_avg_val_loss=0
                while UP<6 and epoch<300:
                    epoch+=1
                    k+=1
                    model.train()
                    for x_batch, y_batch in train_loader:
                        y_pred = model(x_batch)
                        loss = loss_fn(y_pred, y_batch)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                    model.eval()
                    avg_val_loss = 0.
                    for x_batch, y_batch in valid_loader:
                        y_pred = model(x_batch).detach()
                        avg_val_loss += (loss_fn(y_pred, y_batch).item() / 
                                         (len(valid_loader.dataset)))
                    if k > 4:
                        if avg_val_loss> old_avg_val_loss:
                            UP+=1
                        if avg_val_loss<old_avg_val_loss:
                            UP=0
                        old_avg_val_loss=avg_val_loss
                        k=0
                    if avg_val_loss<opt_loss:
                        opt_loss = avg_val_loss
                        opt_epoch = epoch
            
           
                av_opt_epoch+=opt_epoch/n_splits
                av_opt_loss+=opt_loss/n_splits
            print(H1,H2,H3,av_opt_epoch,av_opt_loss)
         


5 5 3 63.166666666666664 2.0161098210016886
5 5 6 38.166666666666664 2.0168591907289293
5 5 9 70.33333333333333 1.7814710505803426
5 5 12 78.5 1.8820731189515856
5 10 3 71.83333333333333 1.8982495856285093
5 10 6 17.833333333333336 2.11842806286282
5 10 9 23.66666666666667 1.9021652295854354
5 10 12 60.33333333333333 1.8315642801920573
5 15 3 32.16666666666667 1.9668986145655312
5 15 6 23.833333333333336 2.135356792343987
5 15 9 21.166666666666668 2.1197585752275256
5 15 12 40.00000000000001 1.7717885531319513
5 20 3 30.833333333333332 2.1502599843343098
5 20 6 85.0 1.9004218350516424
5 20 9 58.0 1.782087189356486
5 20 12 12.999999999999998 2.19539260758294
10 5 3 65.33333333333334 2.0022909635967676
10 5 6 40.99999999999999 2.1700437763002185
10 5 9 19.833333333333332 2.2543581098980376
10 5 12 27.0 2.007557044559055
10 10 3 42.33333333333333 1.7638210116492379
10 10 6 46.666666666666664 1.7627982425689697
10 10 9 23.000000000000004 1.9570418066448636
10 10 12 62.333333333333336 1.757

In [7]:
torch.save(model.state_dict(), ("C:/Users/Gabriel/Documents/Python/ANN3.pt"))                
                                